In [24]:
import os  
import pandas as pd

from scipy.stats import uniform as sp_randFloat
from scipy.stats import randint as sp_randInt

from catboost import Pool, CatBoostClassifier, utils
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split

In [2]:
RANDOM_STATE = 251023

In [3]:
train = pd.read_csv('Datasets/train.csv')
test = pd.read_csv('Datasets/test.csv')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159256 entries, 0 to 159255
Data columns (total 24 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   id                   159256 non-null  int64  
 1   age                  159256 non-null  int64  
 2   height(cm)           159256 non-null  int64  
 3   weight(kg)           159256 non-null  int64  
 4   waist(cm)            159256 non-null  float64
 5   eyesight(left)       159256 non-null  float64
 6   eyesight(right)      159256 non-null  float64
 7   hearing(left)        159256 non-null  int64  
 8   hearing(right)       159256 non-null  int64  
 9   systolic             159256 non-null  int64  
 10  relaxation           159256 non-null  int64  
 11  fasting blood sugar  159256 non-null  int64  
 12  Cholesterol          159256 non-null  int64  
 13  triglyceride         159256 non-null  int64  
 14  HDL                  159256 non-null  int64  
 15  LDL              

In [5]:
train.describe()

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
count,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,...,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000,159256.000000
mean,79627.500000,44.306626,165.266929,67.143662,83.001990,1.005798,1.000989,1.023974,1.023421,122.503648,...,55.852684,114.607682,14.796965,1.074233,0.892764,25.516853,26.550296,36.216004,0.197996,0.437365
std,45973.391572,11.842286,8.818970,12.586198,8.957937,0.402113,0.392299,0.152969,0.151238,12.729315,...,13.964141,28.158931,1.431213,0.347856,0.179346,9.464882,17.753070,31.204643,0.398490,0.496063
min,0.000000,20.000000,135.000000,30.000000,51.000000,0.100000,0.100000,1.000000,1.000000,77.000000,...,9.000000,1.000000,4.900000,1.000000,0.100000,6.000000,1.000000,2.000000,0.000000,0.000000
25%,39813.750000,40.000000,160.000000,60.000000,77.000000,0.800000,0.800000,1.000000,1.000000,114.000000,...,45.000000,95.000000,13.800000,1.000000,0.800000,20.000000,16.000000,18.000000,0.000000,0.000000
50%,79627.500000,40.000000,165.000000,65.000000,83.000000,1.000000,1.000000,1.000000,1.000000,121.000000,...,54.000000,114.000000,15.000000,1.000000,0.900000,24.000000,22.000000,27.000000,0.000000,0.000000
75%,119441.250000,55.000000,170.000000,75.000000,89.000000,1.200000,1.200000,1.000000,1.000000,130.000000,...,64.000000,133.000000,15.800000,1.000000,1.000000,29.000000,32.000000,44.000000,0.000000,1.000000
max,159255.000000,85.000000,190.000000,130.000000,127.000000,9.900000,9.900000,2.000000,2.000000,213.000000,...,136.000000,1860.000000,21.000000,6.000000,9.900000,778.000000,2914.000000,999.000000,1.000000,1.000000


In [19]:
test.head(10)

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,triglyceride,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries
0,159256,40,165,70,84.0,1.2,1.2,1,1,130,...,186,49,115,14.2,1,0.9,19,25,32,0
1,159257,80,160,60,93.0,1.0,1.0,2,2,144,...,158,35,104,13.0,1,1.1,20,12,24,0
2,159258,60,170,70,86.5,0.6,0.7,1,1,117,...,173,39,88,15.4,1,1.4,38,60,36,0
3,159259,40,160,50,67.0,0.3,0.4,1,1,116,...,47,75,128,14.5,1,0.6,25,18,10,1
4,159260,40,170,75,89.4,1.0,0.9,1,1,132,...,100,39,123,16.5,1,1.0,30,39,27,1
5,159261,40,165,55,75.8,1.0,1.0,1,1,94,...,146,41,108,15.1,1,1.0,27,15,56,1
6,159262,40,165,80,87.0,1.5,1.5,1,1,128,...,95,46,160,15.5,1,1.0,24,30,25,0
7,159263,55,150,50,69.2,1.0,0.9,1,1,152,...,89,60,98,14.2,1,0.7,26,20,36,0
8,159264,70,160,60,79.0,0.5,0.5,1,1,150,...,116,57,113,13.9,1,1.2,19,24,18,0
9,159265,70,145,55,82.0,1.0,1.0,1,1,144,...,72,49,110,14.5,1,1.0,20,15,27,0


In [6]:
train.head(10)

,id,age,height(cm),weight(kg),waist(cm),eyesight(left),eyesight(right),hearing(left),hearing(right),systolic,...,HDL,LDL,hemoglobin,Urine protein,serum creatinine,AST,ALT,Gtp,dental caries,smoking
0,0,55,165,60,81.0,0.5,0.6,1,1,135,...,40,75,16.5,1,1.0,22,25,27,0,1
1,1,70,165,65,89.0,0.6,0.7,2,2,146,...,57,126,16.2,1,1.1,27,23,37,1,0
2,2,20,170,75,81.0,0.4,0.5,1,1,118,...,45,93,17.4,1,0.8,27,31,53,0,1
3,3,35,180,95,105.0,1.5,1.2,1,1,131,...,38,102,15.9,1,1.0,20,27,30,1,0
4,4,30,165,60,80.5,1.5,1.0,1,1,121,...,44,93,15.4,1,0.8,19,13,17,0,1
5,5,50,170,55,51.0,1.2,1.2,1,1,146,...,31,99,15.9,1,0.7,24,42,119,1,1
6,6,45,160,55,69.0,1.5,1.2,1,1,150,...,69,122,13.0,1,0.7,17,12,16,0,0
7,7,55,155,60,84.5,0.7,0.9,1,1,137,...,51,198,14.5,1,0.7,16,15,16,0,0
8,8,40,165,70,89.0,0.7,1.0,1,1,130,...,59,150,15.7,1,0.9,24,21,31,0,1
9,9,40,155,50,73.0,1.5,1.5,1,1,105,...,55,122,13.2,1,0.7,22,16,14,0,0


In [7]:
features = train.drop('smoking', axis=1)
target = train['smoking']
cat_features = ['HDL', 'LDL', 'AST', 'ALT', 'dental caries']

In [8]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2)

In [9]:
cbc = CatBoostClassifier(
    iterations=100,
    random_state=RANDOM_STATE,
    cat_features=cat_features,
    custom_loss='AUC:hints=skip_train~false',
    loss_function='CrossEntropy',
    eval_metric='AUC:hints=skip_train~false',
    early_stopping_rounds=40
)

In [10]:
cbc_parm_grid = {'depth' : sp_randInt(1, 10),
                 'learning_rate' : sp_randFloat(),
                 'iterations' : sp_randInt(10, 100),
                 'l2_leaf_reg' : sp_randInt(1, 5),
                 'border_count' : sp_randInt(32, 255)
                }

In [11]:
cbc.randomized_search(cbc_parm_grid,
                  features_train,
                  y=target_train,
                  cv=5,
                  n_iter=50,
                  partition_random_seed=RANDOM_STATE,
                  calc_cv_statistics=True,
                  search_by_train_test_split=False,
                  refit=True,
                  shuffle=True,
                  stratified=None,
                  train_size=1,
                  verbose=True
                     )

Training on fold [0/5]
0:	learn: 0.8164943	test: 0.8138813	best: 0.8138813 (0)	total: 81.1ms	remaining: 4.54s
1:	learn: 0.8312749	test: 0.8273511	best: 0.8273511 (1)	total: 105ms	remaining: 2.9s
2:	learn: 0.8402415	test: 0.8357703	best: 0.8357703 (2)	total: 125ms	remaining: 2.26s
3:	learn: 0.8442828	test: 0.8403028	best: 0.8403028 (3)	total: 146ms	remaining: 1.93s
4:	learn: 0.8478589	test: 0.8436204	best: 0.8436204 (4)	total: 181ms	remaining: 1.88s
5:	learn: 0.8501947	test: 0.8455856	best: 0.8455856 (5)	total: 209ms	remaining: 1.78s
6:	learn: 0.8511802	test: 0.8469168	best: 0.8469168 (6)	total: 237ms	remaining: 1.69s
7:	learn: 0.8524981	test: 0.8479273	best: 0.8479273 (7)	total: 266ms	remaining: 1.63s
8:	learn: 0.8537715	test: 0.8484977	best: 0.8484977 (8)	total: 296ms	remaining: 1.58s
9:	learn: 0.8552386	test: 0.8499653	best: 0.8499653 (9)	total: 327ms	remaining: 1.54s
10:	learn: 0.8562914	test: 0.8511943	best: 0.8511943 (10)	total: 356ms	remaining: 1.49s
11:	learn: 0.8569177	test: 0.

42:	learn: 0.8703929	test: 0.8580020	best: 0.8582576 (35)	total: 1.34s	remaining: 436ms
43:	learn: 0.8707735	test: 0.8582759	best: 0.8582759 (43)	total: 1.37s	remaining: 405ms
44:	learn: 0.8710577	test: 0.8580762	best: 0.8582759 (43)	total: 1.41s	remaining: 375ms
45:	learn: 0.8713361	test: 0.8582035	best: 0.8582759 (43)	total: 1.44s	remaining: 344ms
46:	learn: 0.8714711	test: 0.8582532	best: 0.8582759 (43)	total: 1.47s	remaining: 313ms
47:	learn: 0.8715681	test: 0.8582653	best: 0.8582759 (43)	total: 1.5s	remaining: 282ms
48:	learn: 0.8717477	test: 0.8582174	best: 0.8582759 (43)	total: 1.54s	remaining: 251ms
49:	learn: 0.8719442	test: 0.8581251	best: 0.8582759 (43)	total: 1.57s	remaining: 220ms
50:	learn: 0.8720928	test: 0.8580693	best: 0.8582759 (43)	total: 1.6s	remaining: 188ms
51:	learn: 0.8722842	test: 0.8579330	best: 0.8582759 (43)	total: 1.64s	remaining: 157ms
52:	learn: 0.8724868	test: 0.8576837	best: 0.8582759 (43)	total: 1.67s	remaining: 126ms
53:	learn: 0.8726828	test: 0.85776

24:	learn: 0.8636190	test: 0.8606843	best: 0.8608543 (23)	total: 710ms	remaining: 909ms
25:	learn: 0.8639699	test: 0.8606946	best: 0.8608543 (23)	total: 742ms	remaining: 885ms
26:	learn: 0.8644110	test: 0.8605302	best: 0.8608543 (23)	total: 771ms	remaining: 857ms
27:	learn: 0.8647241	test: 0.8602816	best: 0.8608543 (23)	total: 809ms	remaining: 837ms
28:	learn: 0.8651693	test: 0.8603789	best: 0.8608543 (23)	total: 834ms	remaining: 805ms
29:	learn: 0.8653971	test: 0.8603544	best: 0.8608543 (23)	total: 866ms	remaining: 779ms
30:	learn: 0.8657090	test: 0.8604670	best: 0.8608543 (23)	total: 892ms	remaining: 748ms
31:	learn: 0.8661377	test: 0.8605333	best: 0.8608543 (23)	total: 931ms	remaining: 728ms
32:	learn: 0.8664758	test: 0.8605361	best: 0.8608543 (23)	total: 966ms	remaining: 702ms
33:	learn: 0.8667796	test: 0.8609472	best: 0.8609472 (33)	total: 996ms	remaining: 674ms
34:	learn: 0.8670318	test: 0.8610070	best: 0.8610070 (34)	total: 1.02s	remaining: 644ms
35:	learn: 0.8673227	test: 0.861

3:	learn: 0.8459085	test: 0.8409311	best: 0.8409311 (3)	total: 121ms	remaining: 997ms
4:	learn: 0.8482214	test: 0.8430894	best: 0.8430894 (4)	total: 149ms	remaining: 953ms
5:	learn: 0.8515400	test: 0.8463252	best: 0.8463252 (5)	total: 180ms	remaining: 932ms
6:	learn: 0.8530754	test: 0.8476279	best: 0.8476279 (6)	total: 212ms	remaining: 910ms
7:	learn: 0.8543632	test: 0.8489643	best: 0.8489643 (7)	total: 256ms	remaining: 927ms
8:	learn: 0.8558960	test: 0.8503796	best: 0.8503796 (8)	total: 284ms	remaining: 884ms
9:	learn: 0.8569798	test: 0.8513586	best: 0.8513586 (9)	total: 311ms	remaining: 838ms
10:	learn: 0.8577244	test: 0.8518414	best: 0.8518414 (10)	total: 341ms	remaining: 805ms
11:	learn: 0.8585254	test: 0.8527065	best: 0.8527065 (11)	total: 365ms	remaining: 760ms
12:	learn: 0.8591316	test: 0.8526406	best: 0.8527065 (11)	total: 388ms	remaining: 717ms
13:	learn: 0.8596651	test: 0.8530328	best: 0.8530328 (13)	total: 413ms	remaining: 679ms
14:	learn: 0.8603978	test: 0.8533771	best: 0.8

27:	learn: 0.8685596	test: 0.8604704	best: 0.8604744 (21)	total: 726ms	remaining: 233ms
28:	learn: 0.8690446	test: 0.8606396	best: 0.8606396 (28)	total: 751ms	remaining: 207ms
29:	learn: 0.8694981	test: 0.8603986	best: 0.8606396 (28)	total: 776ms	remaining: 181ms
30:	learn: 0.8701117	test: 0.8602635	best: 0.8606396 (28)	total: 803ms	remaining: 155ms
31:	learn: 0.8703992	test: 0.8602784	best: 0.8606396 (28)	total: 827ms	remaining: 129ms
32:	learn: 0.8708658	test: 0.8601005	best: 0.8606396 (28)	total: 852ms	remaining: 103ms
33:	learn: 0.8712902	test: 0.8601807	best: 0.8606396 (28)	total: 876ms	remaining: 77.3ms
34:	learn: 0.8716388	test: 0.8602550	best: 0.8606396 (28)	total: 899ms	remaining: 51.4ms
35:	learn: 0.8721217	test: 0.8598591	best: 0.8606396 (28)	total: 922ms	remaining: 25.6ms
36:	learn: 0.8724951	test: 0.8599752	best: 0.8606396 (28)	total: 948ms	remaining: 0us

bestTest = 0.8606396044
bestIteration = 28

Training on fold [3/5]
0:	learn: 0.8177605	test: 0.8212342	best: 0.8212342

...

7:	learn: 0.8233548	test: 0.8229358	best: 0.8229358 (7)	total: 124ms	remaining: 855ms
8:	learn: 0.8254095	test: 0.8249124	best: 0.8249124 (8)	total: 143ms	remaining: 857ms
9:	learn: 0.8274276	test: 0.8267912	best: 0.8267912 (9)	total: 161ms	remaining: 852ms
10:	learn: 0.8274475	test: 0.8266131	best: 0.8267912 (9)	total: 177ms	remaining: 838ms
11:	learn: 0.8288066	test: 0.8277612	best: 0.8277612 (11)	total: 193ms	remaining: 820ms
12:	learn: 0.8297630	test: 0.8288115	best: 0.8288115 (12)	total: 209ms	remaining: 803ms
13:	learn: 0.8311891	test: 0.8300228	best: 0.8300228 (13)	total: 225ms	remaining: 788ms
14:	learn: 0.8329011	test: 0.8320658	best: 0.8320658 (14)	total: 240ms	remaining: 768ms
15:	learn: 0.8334260	test: 0.8323991	best: 0.8323991 (15)	total: 256ms	remaining: 751ms
16:	learn: 0.8345106	test: 0.8338404	best: 0.8338404 (16)	total: 271ms	remaining: 733ms
17:	learn: 0.8360858	test: 0.8355339	best: 0.8355339 (17)	total: 286ms	remaining: 714ms
18:	learn: 0.8369602	test: 0.8363004	be

44:	learn: 0.8506438	test: 0.8505962	best: 0.8505962 (44)	total: 727ms	remaining: 291ms
45:	learn: 0.8507397	test: 0.8506639	best: 0.8506639 (45)	total: 746ms	remaining: 276ms
46:	learn: 0.8511803	test: 0.8511799	best: 0.8511799 (46)	total: 766ms	remaining: 261ms
47:	learn: 0.8515373	test: 0.8514517	best: 0.8514517 (47)	total: 783ms	remaining: 245ms
48:	learn: 0.8516370	test: 0.8515745	best: 0.8515745 (48)	total: 798ms	remaining: 228ms
49:	learn: 0.8516639	test: 0.8516457	best: 0.8516457 (49)	total: 815ms	remaining: 212ms
50:	learn: 0.8517963	test: 0.8517439	best: 0.8517439 (50)	total: 830ms	remaining: 195ms
51:	learn: 0.8519597	test: 0.8518909	best: 0.8518909 (51)	total: 846ms	remaining: 179ms
52:	learn: 0.8521598	test: 0.8518985	best: 0.8518985 (52)	total: 861ms	remaining: 162ms
53:	learn: 0.8521537	test: 0.8518961	best: 0.8518985 (52)	total: 877ms	remaining: 146ms
54:	learn: 0.8522802	test: 0.8519310	best: 0.8519310 (54)	total: 894ms	remaining: 130ms
55:	learn: 0.8523882	test: 0.852

20:	learn: 0.8412569	test: 0.8344715	best: 0.8344715 (20)	total: 350ms	remaining: 701ms
21:	learn: 0.8428238	test: 0.8358821	best: 0.8358821 (21)	total: 369ms	remaining: 688ms
22:	learn: 0.8437354	test: 0.8369963	best: 0.8369963 (22)	total: 385ms	remaining: 669ms
23:	learn: 0.8443719	test: 0.8375019	best: 0.8375019 (23)	total: 400ms	remaining: 650ms
24:	learn: 0.8446825	test: 0.8382802	best: 0.8382802 (24)	total: 416ms	remaining: 632ms
25:	learn: 0.8451764	test: 0.8391506	best: 0.8391506 (25)	total: 432ms	remaining: 615ms
26:	learn: 0.8454945	test: 0.8393207	best: 0.8393207 (26)	total: 448ms	remaining: 597ms
27:	learn: 0.8455486	test: 0.8394100	best: 0.8394100 (27)	total: 464ms	remaining: 580ms
28:	learn: 0.8460012	test: 0.8398003	best: 0.8398003 (28)	total: 479ms	remaining: 562ms
29:	learn: 0.8461641	test: 0.8400845	best: 0.8400845 (29)	total: 494ms	remaining: 544ms
30:	learn: 0.8466261	test: 0.8408445	best: 0.8408445 (30)	total: 511ms	remaining: 527ms
31:	learn: 0.8471127	test: 0.841

4:	learn: 0.8153581	test: 0.8152009	best: 0.8152009 (4)	total: 88.5ms	remaining: 319ms
5:	learn: 0.8171395	test: 0.8175124	best: 0.8175124 (5)	total: 114ms	remaining: 323ms
6:	learn: 0.8170056	test: 0.8176118	best: 0.8176118 (6)	total: 132ms	remaining: 301ms
7:	learn: 0.8179024	test: 0.8183262	best: 0.8183262 (7)	total: 151ms	remaining: 283ms
8:	learn: 0.8194544	test: 0.8197342	best: 0.8197342 (8)	total: 170ms	remaining: 265ms
9:	learn: 0.8203830	test: 0.8205903	best: 0.8205903 (9)	total: 187ms	remaining: 243ms
10:	learn: 0.8206798	test: 0.8211075	best: 0.8211075 (10)	total: 203ms	remaining: 222ms
11:	learn: 0.8215509	test: 0.8218634	best: 0.8218634 (11)	total: 220ms	remaining: 202ms
12:	learn: 0.8219574	test: 0.8223095	best: 0.8223095 (12)	total: 238ms	remaining: 183ms
13:	learn: 0.8224556	test: 0.8229102	best: 0.8229102 (13)	total: 259ms	remaining: 166ms
14:	learn: 0.8248302	test: 0.8254756	best: 0.8254756 (14)	total: 280ms	remaining: 149ms
15:	learn: 0.8251771	test: 0.8256057	best: 

{'params': {'border_count': 98,
  'depth': 6.0,
  'learning_rate': 0.33621297119959015,
  'l2_leaf_reg': 4.0,
  'iterations': 96.0},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
             

In [12]:
cbc.best_score_

{'learn': {'CrossEntropy': 0.4302927659996272, 'AUC': 0.8758579140370799}}

In [13]:
predictions = cbc.predict_proba(features_test)

In [14]:
roc_auc_score(target_test, predictions[:,1])

0.8657142502027709

In [20]:
cbc.fit(features, target)
test_proba = cbc.predict_proba(test)

0:	learn: 0.8216042	total: 27.2ms	remaining: 2.58s
1:	learn: 0.8313705	total: 56.2ms	remaining: 2.64s
2:	learn: 0.8363668	total: 86.6ms	remaining: 2.69s
3:	learn: 0.8410595	total: 115ms	remaining: 2.63s
4:	learn: 0.8437397	total: 144ms	remaining: 2.62s
5:	learn: 0.8459404	total: 174ms	remaining: 2.6s
6:	learn: 0.8472494	total: 207ms	remaining: 2.63s
7:	learn: 0.8499122	total: 233ms	remaining: 2.56s
8:	learn: 0.8515629	total: 264ms	remaining: 2.56s
9:	learn: 0.8532001	total: 292ms	remaining: 2.52s
10:	learn: 0.8540284	total: 321ms	remaining: 2.48s
11:	learn: 0.8549094	total: 353ms	remaining: 2.47s
12:	learn: 0.8558233	total: 382ms	remaining: 2.44s
13:	learn: 0.8566729	total: 412ms	remaining: 2.42s
14:	learn: 0.8573284	total: 443ms	remaining: 2.39s
15:	learn: 0.8582093	total: 474ms	remaining: 2.37s
16:	learn: 0.8587687	total: 503ms	remaining: 2.34s
17:	learn: 0.8594093	total: 533ms	remaining: 2.31s
18:	learn: 0.8597314	total: 563ms	remaining: 2.28s
19:	learn: 0.8602843	total: 591ms	remai

In [21]:
answer = {
    'id' : test['id'],
    'smoking' : test_proba[:,1]
}

In [27]:
asnwer_df = pd.DataFrame(data=answer)

TypeError: __init__() got an unexpected keyword argument 'reset_index'

In [23]:
asnwer_df

,id,smoking
0,159256,0.585519
1,159257,0.259658
2,159258,0.405124
3,159259,0.019701
4,159260,0.550949
...,...,...
106166,265422,0.608719
106167,265423,0.613440
106168,265424,0.478857
106169,265425,0.137006


In [28]:
os.makedirs('result/', exist_ok=True)  
asnwer_df.to_csv('result/out.csv', index=False)  